In [1]:
from email.mime.multipart import MIMEMultipart
import os.path
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import datetime as dt
from email.mime.text import MIMEText
import base64

In [5]:
# Connect to Drive API
spreadsheet_id = '1-2FvSx-rVma1RCZ0Hu4p7J8DJjN6TpZwz6CgNSwa7CY'
url = 'https://docs.google.com/spreadsheets/d/1-2FvSx-rVma1RCZ0Hu4p7J8DJjN6TpZwz6CgNSwa7CY'
emails = ['treasury@celsius.network',
'weekly.portfolio@celsius.network']

SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = None
creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Call the Drive v3 API
SAMPLE_RANGE_NAME = 'Summary Distribution!B2:E210'
request = service.spreadsheets().values().batchGet(
    spreadsheetId=spreadsheet_id,
    ranges=SAMPLE_RANGE_NAME,
    valueRenderOption='FORMATTED_VALUE')
response = request.execute()

In [5]:
'''SCOPES = ['https://www.googleapis.com/auth/gmail.send']
flow = InstalledAppFlow.from_client_secrets_file(
    'client_gmail.json', SCOPES)
creds = flow.run_local_server(port=0)
# Save the credentials for the next run
with open('token_gmail_send.json', 'w') as token:
    token.write(creds.to_json())'''

"SCOPES = ['https://www.googleapis.com/auth/gmail.send']\nflow = InstalledAppFlow.from_client_secrets_file(\n    'client_gmail.json', SCOPES)\ncreds = flow.run_local_server(port=0)\n# Save the credentials for the next run\nwith open('token_gmail_send.json', 'w') as token:\n    token.write(creds.to_json())"

In [68]:
df = pd.DataFrame(response['valueRanges'][0]['values']).dropna().reset_index(drop=True)
df.set_index(df[0], inplace=True)
df.columns = df.iloc[0]

summary_idx = [
        'Deployable Coin Metric',
        'Deployable Liquidity Reserve (Tiers 1-4)',
        'Non-Stablecoins',
        'BTC',
        'ETH',
        'Non-Stablecoins (All Other)',
        'Stablecoins¹',
        'Gold Tokens',
        'Deployable Liquidity Excess (Tier 5+)',
        'Total Undeployed',
        'Total Under Deployed']

df_tiers1_4 = df.loc[summary_idx[0]:summary_idx[8]]
df_tiers1_4.index = df_tiers1_4.index.str.strip()
df_tiers1_4 = df_tiers1_4.loc[summary_idx[0:8]]
df_tier5  = df.loc[summary_idx[8:]]
df_all = pd.concat([df_tiers1_4,df_tier5])
df_all.drop([''],axis=1, inplace=True)

#Format for mail
df_all_t = df_all.copy()
sub_col1 = ['',''] + summary_idx[2:8] + ['','','']
#['','Deployable Liquidity Reserve (Tiers 1-4)'] + ['' for i in range(6)] +summary_idx[-3:]
df_all_t.loc[summary_idx[2:8], 'Deployable Coin Metric'] = ''
df_all_t.insert(1,'-', sub_col1)
df_all_t = df_all_t.iloc[1:]
df_all_t.rename(columns={'Deployable Coin Metric':'--'}, inplace=True)

In [69]:
# Connect to GMAIL API
client_id = "me"
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
creds = None
creds = Credentials.from_authorized_user_file('token_gmail_send.json', SCOPES)

service_gmail = build('gmail', 'v1', credentials=creds)
#results = service_gmail.users().labels().list(userId=client_id).execute()
#labels = results.get('labels', [])

In [71]:
html = """
<html><body><p>Hi all,</p>
<p>here is a snapshot of the Deployable Coin Metric:</p>
{table}

<p> And the link to the google sheet:</p>
<a href="{url}">Deployable Metric</a>

<p>Regards,</p>
<p>CNS BOT</p>
</body></html>
"""

user_id = 'me'
subject = 'Daily Deployable Coin Metric'
to = 'gabriel.choukroun@celsius.network'
sender = 'gabriel.choukroun@celsius.network'

html = html.format(table=df_all_t.to_html(index=False, header=True), url=url)

message = MIMEText(html, 'html')
message['to'] = to
message['from'] = sender
message['subject'] = subject
raw_message = {
    'raw':base64.urlsafe_b64encode(message.as_string().encode("utf-8")).decode("utf-8")}
 #   'payload': {'mimeType': 'text/html'}}

message = service_gmail.users().messages().send(userId=user_id, body=raw_message).execute()
print('Message Id: %s' % message['id'])

Message Id: 17e4e5f685e1f8af
